In [25]:
#모듈 설정
import pandas as pd
import os
import json
from ast import literal_eval

#해상도 줄이기
def all_point_operation(n):
    return round(n*(1024/3024),2)

#작업할 환경
#change
root_path='path'
img_path=root_path+'image folder path'

#csv파일을 데이터프레임으로
#change
df=pd.read_csv(root_path+'csv file', encoding="CP949")

# 빈 딕셔너리 생성
dic={}

    
for i in range(0,len(df)):
    
        file_name=df.values[i][0]+'.jpg'
        #change
        size=os.stat(img_path+file_name).st_size
        file_full_name=file_name+str(size)
        
        # 이미 만들었을 때
        if file_full_name in dic.keys():
            pass
        
        # 처음 만들때
        else:
            #문자열 형식의 json을 리스트 형으로 변환 - 원본 리스트형 좌표 정보
            all_point_target=(literal_eval(df.values[i][1])['segmentation'][0:8])
            #해상도 함수 호출
            all_point_result=list(map(all_point_operation, all_point_target))
        
            dic[file_full_name]={
            'fileref':'',
            'size':size,
            'filename':file_name,
            'base64_img_data':'',
            'file_attributes':{}
            }
            regions_value={}
            for x in range(0,int(df[df['fname']==df.values[i][0]].count()[0])):
                
                    all_point_target=(literal_eval(df[df['fname']==df.values[i][0]].values[x][1])['segmentation'][0:8])
                    all_point_result=list(map(all_point_operation, all_point_target))
                    regions_value[str(x)]={
                    "shape_attributes":{
                    'name':'polygon',
                    'all_points_x':all_point_result[::2],
                    'all_points_y':all_point_result[1::2]
                    },
                    'region_attributes':{}
                    }
                    
            dic[file_full_name]['regions']=regions_value
        
            
            

        
with open(root_path+'result.json','a') as make_file:
    json.dump(dic, make_file, indent='\t')

In [26]:
#잘 생성되었는지 체크
#각 모델별 사용가능한 이미지의 개수
with open(root_path+'result.json','r') as read_file:
    json_data=json.load(read_file)
len(json_data)

142

In [27]:
# 추출한 json 데이터와 실제 폴더 내 이미지 정확하게 맞추기
with open(root_path+'result.json','r') as read_file:
    json_data=json.load(read_file)

dict={}

for i in os.listdir(img_path):
    
    size=os.stat(img_path+i).st_size
    file_name=i+str(size)

    if file_name in json_data.keys():
        pass
    else:
        os.remove(img_path+i)

In [28]:
#train, val, submisson 폴더 생성
def createFolder(directory):
    
    #같은 이름의 폴더가 없으면 생성
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
createFolder(root_path+'train')
createFolder(root_path+'val')
createFolder(root_path+'submission')

train_path=root_path+'train/'
val_path=root_path+'val/'
submission_path=root_path+'submission/'

In [29]:
# 이미지들을 train, val, submission 폴더에 각각 배치
# train, val 4:1 비율, submission 5개 고정
import random
import shutil

def submission():

    files=os.listdir(img_path)
    
    for i in random.sample(files, 5):
        shutil.move(img_path+i, submission_path+i)

def train():

    files=os.listdir(img_path)
    train_no=round(len(files)*(4/5))
    
    for i in random.sample(files, train_no):
        shutil.move(img_path+i, train_path+i)

def val():

    files=os.listdir(img_path)
    train_no=round(len(files)*(4/5))
    
    for i in files:
        shutil.move(img_path+i, val_path+i)
        
submission()
train()
val()

In [30]:
# train,val json 생성
def train_json():
    
    with open(root_path+'result.json','r') as read_file:
        json_data=json.load(read_file)
    
    dict={}
    files=os.listdir(train_path)
    
    for i in files:
        
        size=os.stat(train_path+i).st_size
        file_full_name=i+str(size)
        
        dict[file_full_name]=json_data[file_full_name]
    
    with open(train_path+'via_region_data.json','a') as make_file:
        json.dump(dict, make_file, indent='\t')

def val_json():
    
    with open(root_path+'result.json','r') as read_file:
        json_data=json.load(read_file)
    
    dict={}
    files=os.listdir(val_path)
    
    for i in files:
        
        size=os.stat(val_path+i).st_size
        file_full_name=i+str(size)
        
        dict[file_full_name]=json_data[file_full_name]
    
    with open(val_path+'via_region_data.json','a') as make_file:
        json.dump(dict, make_file, indent='\t')
        
train_json()
val_json()